## Siamese Neural Networks 
### for Supervised Clustering of High Dimensional Spaces

In [1]:
!python -V
!which python

Python 3.7.10
/Users/michellemacrae/Documents/Sean/mobius/.venv/bin/python


In [2]:
import pandas as pd

# DOWNLOAD DATSET HERE: https://www.kaggle.com/nehaprabhavalkar/av-healthcare-analytics-ii
los = pd.read_csv("data/train_data.csv")

print(los.shape)
los.head()

(318438, 18)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [3]:
# import seaborn as sns

# sns.pairplot(los, hue='Stay')

In [4]:
from sklearn.model_selection import train_test_split

los_sample, _ = train_test_split(
    los,
    test_size=0.99,
    stratify=los["Stay"])

los_sample.shape

(3184, 18)

## Tabular Learner

Before we train the Tabular Siamese Learner we will train baseline Tabular Learner for species classification... (why do we do this, exactly? can we just instantiate a Tabular Siamese Learner without a baseline Tabular Learner ???)

Ah yes, to init a new `TabularSiameseModel` we need to provide an `encoder` and `head` and the Tabular Learner will act as the `encoder` we init the `TabularSiameseModel` with.

In [5]:
from sklearn.model_selection import train_test_split

# los_train, los_val = train_test_split(
#     los,
#     test_size=0.10,
#     stratify=los["Stay"])

los_train, los_val = train_test_split(
    los_sample,
    test_size=0.10,
    stratify=los_sample["Stay"])

los_train.shape, los_val.shape

df = los_train.copy()

exclude_vars = ["case_id", "patientid"]

In [6]:
from fastai.tabular.all import CategoryBlock
                                
# y_names = ["species"]
y_names = ["Stay"]
y_block = CategoryBlock()

In [7]:
from mobius.utils import emb_sz_rule

cat_names = [x for x in df.select_dtypes(exclude=['int', 'float']).columns if x != y_names]
cat_names = [x for x in cat_names if x not in exclude_vars]

# calc embedding sizes for each categorical feature
emb_szs = {k: emb_sz_rule(len(df[k].unique())) for k in cat_names}
emb_szs

{'Hospital_type_code': 5,
 'Hospital_region_code': 3,
 'Department': 4,
 'Ward_Type': 4,
 'Ward_Facility_Code': 4,
 'Type of Admission': 3,
 'Severity of Illness': 3,
 'Age': 6,
 'Stay': 6}

In [8]:
import numpy as np

cont_names = [x for x in df.select_dtypes([np.number]).columns if x != y_names]
cont_names = [x for x in cont_names if x not in exclude_vars]
cont_names

['Hospital_code',
 'City_Code_Hospital',
 'Available Extra Rooms in Hospital',
 'Bed Grade',
 'City_Code_Patient',
 'Visitors with Patient',
 'Admission_Deposit']

In [9]:
from fastai.tabular.all import (Categorify, CategoryBlock, FillMissing,
                                Normalize, TabDataLoader, TabularPandas,
                                tabular_config, tabular_learner)

procs = [FillMissing, Categorify, Normalize]

In [10]:
from fastai.data.core import range_of
from fastai.tabular.all import RandomSplitter

# train/test split
splits = RandomSplitter(valid_pct=0.10)(range_of(df))

In [11]:
tabular_pandas = TabularPandas(
        df,
        procs=procs,
        cat_names=cat_names,
        cont_names=cont_names,
        y_names=y_names,
        y_block=y_block,
        splits=splits,
        device="cpu")

In [12]:
trn_dl = TabDataLoader(
    tabular_pandas.train,
    bs=128,
    shuffle=True,
    drop_last=True,
    num_workers=4)

val_dl = TabDataLoader(
    tabular_pandas.valid,
    bs=128,
    num_workers=4)

In [13]:
from fastai.data.core import DataLoaders

dls = DataLoaders(trn_dl, val_dl)

print("Sample batch:")
# dls.one_batch()

Sample batch:


In [14]:
from fastai.metrics import F1Score, Precision, Recall, accuracy

# load the tabular_pandas data through the tabular_learner
layers = [2048, 1024, 32, 2]

# tabular learner configuration
config = tabular_config(ps=[0.2, 0.1, 0.1, 0.0], embed_p=0.1)

learn = tabular_learner(
    dls,
    layers=layers,
    emb_szs=emb_szs,
    config=config,
    metrics=[accuracy,
             Precision(average='macro'),
             Recall(average='macro'),
             F1Score(average='macro')])

In [15]:
learn.fit_one_cycle(n_epoch=2)

epoch,train_loss,valid_loss,accuracy,precision_score,recall_score,f1_score,time
0,2.226802,2.430841,0.227273,0.042744,0.120777,0.061860,00:02
1,2.092716,2.263538,0.209790,0.148267,0.183682,0.130910,00:01


/Users/michellemacrae/Documents/Sean/mobius/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michellemacrae/Documents/Sean/mobius/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# learn.export("tabular_learn.pkl")

In [17]:
from mobius.calibration import ModelWithTemperature

scaled_model = ModelWithTemperature(learn.model)
scaled_model.set_temperature(val_dl)
learn.model = scaled_model.model

Before temperature - NLL: 2.264, ECE: 0.104
Optimal temperature: 0.932
After temperature - NLL: 2.261, ECE: 0.128


In [18]:
# true species labels
y_true=learn.dls.valid.items["Stay"]

# model scores and species predictions
y_scores, *_ = learn.get_preds(dl=val_dl)
preds = np.argmax(y_scores, 1).numpy()

/Users/michellemacrae/Documents/Sean/mobius/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
print("First 20 species labels and predictions")
list(zip(y_true, preds))[:10]

First 20 species labels and predictions


[(0, 4),
 (7, 4),
 (1, 4),
 (3, 4),
 (3, 4),
 (1, 4),
 (7, 4),
 (3, 4),
 (1, 2),
 (3, 4)]

In [20]:
(y_true == preds).sum() / len(y_true)

0.2097902097902098

## Siamese Net

To init a new `TabularSiameseDataset` object, we only need a `tabular_pandas` object from the fast.ai library.

In [21]:
from mobius.datasets import write_jsonl

# write SNN training data to `data/`
write_jsonl(tabular_pandas.train.to.items[0].items, "data/train_data.jsonl")
write_jsonl(tabular_pandas.valid.to.items[0].items, "data/valid_data.jsonl")

# write SNN training labels to `data/`
tabular_pandas.train.y.to_csv("data/train_labels.csv", index=True)
tabular_pandas.valid.y.to_csv("data/valid_labels.csv", index=True)

In [22]:
from mobius.datasets import TabularSiameseDataset

train_ds = TabularSiameseDataset(
    csv_file="data/train_labels.csv", 
    jsonl_file="data/train_data.jsonl",
    tabular_learner=learn)
    
valid_ds = TabularSiameseDataset(
    csv_file="data/valid_labels.csv", 
    jsonl_file="data/valid_data.jsonl",
    tabular_learner=learn)

In [23]:
train_ds.__len__(), train_ds.__getitem__(0)

(2579,
 ((tensor([1, 1, 3, 2, 6, 3, 2, 2, 3, 1, 1]),
   tensor([ 0.5247,  0.4156,  3.2945,  1.6370, -1.3029,  2.6328, -1.7517])),
  (tensor([2, 2, 2, 1, 4, 2, 1, 6, 3, 1, 1]),
   tensor([ 0.8730, -0.8915,  0.7167,  0.4694, -0.0743,  0.4011, -0.7560])),
  tensor(0.)))

In [24]:
valid_ds.__len__(), valid_ds.__getitem__(0)

(286,
 ((tensor([1, 1, 3, 2, 6, 3, 2, 2, 3, 1, 1]),
   tensor([ 0.5247,  0.4156,  3.2945,  1.6370, -1.3029,  2.6328, -1.7517])),
  (tensor([1, 2, 3, 4, 3, 1, 1, 6, 5, 1, 1]),
   tensor([ 0.0603,  0.7424, -1.0019, -0.6981,  0.1304,  1.5169, -0.2078])),
  tensor(1.)))

In [25]:
dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=16, device='cpu', num_workers=4)

Siamese net encoder is the body of the Tabular net we just trained.

In [26]:
dls.dataset.c

11

In [27]:
import copy

encoder = copy.copy(learn)
encoder.model.layers = learn.model.layers[:-1]
encoder_model = encoder.model

We create a new head that doubles the input shape to the last layer of the trained Tabular net, since the loss function will now compare 2 penguins. The size of the output shape is set by...???

In [28]:
from fastai.layers import LinBnDrop

head = LinBnDrop(n_in=layers[-1]*2,
    n_out=16,  # size of output space
    bn=True,
    act=None)

In [29]:
import time
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from fastai.callback.core import Callback


# TODO: save every TSNE model as pkl - interactive 3-d w/ tooltip to show point information (id, features, etc.)
class TSNECallback(Callback):
    def after_validate(self):
        plt.clf()
        t = int(time.time())
        train_encoded = list()
        for i in range(len(siamese_learner.dls.valid.dataset.labels)):
            p, _, _ = siamese_learner.dls.valid_ds.__getitem__(i)

            # rehsape into mini-batch size 1
            p = p[0].reshape(1, -1), p[1].reshape(1, -1)

            # encode the household into output embedding space
            p_encode = siamese_learner.model.encode(p)
            train_encoded.append(p_encode)
            
        ids = self.dls.valid.get_idxs()
        y_train_labels = self.dls.valid.dataset.labels["Stay"]
        train_encoded_df = pd.DataFrame(torch.stack(train_encoded).squeeze())

        # write encoded space to csv
        train_encoded_df.to_csv(f"tsne_{t}_{self.epoch}.csv")
        
        # limit the permutation of hyper-params of t-SNE
        tsne = TSNE(n_components=2, metric="euclidean", n_iter=500)
        encoded_train_tsne = tsne.fit_transform(train_encoded_df.values)
        np.save(f"tsne_{t}_{self.epoch}.npy", encoded_train_tsne)
        
        # TODO: add title...
        sns.scatterplot(x=encoded_train_tsne[:,0],
                        y=encoded_train_tsne[:,1],
                        hue=y_train_labels, 
                        legend=False, 
                        palette="tab10").figure.savefig(f"snn_{t}_epoch_{self.epoch}.png")
        
        
# TODO: run knn on embedding space to show learned embedding is informative!!!

In [30]:
from mobius.models import TabularSiameseModel

model = TabularSiameseModel(encoder_model, head)

In [31]:
from fastai.torch_basics import params
from mobius.losses import ContrastiveLoss

def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]

def contrastive_loss_func(out, targ):
    return ContrastiveLoss(margin=0.20)(out, targ.long())

In [32]:
from fastai.learner import Learner

siamese_learner = Learner(dls,
    model,
    model_dir=".",
    loss_func=contrastive_loss_func,
    splitter=siamese_splitter,
    cbs=[TSNECallback])

In [33]:
# y_labels = siamese_learner.dls.valid.dataset.labels["Stay"]
# y_labels.to_csv("y_labels.csv")

In [34]:
# %debug
siamese_learner.summary()

TabularSiameseModel (Input shape: ["['16 x 11', '16 x 7']", "['16 x 11', '16 x 7']"])
Layer (type)         Output Shape         Param #    Trainable 
                     16 x 5              
Embedding                                 40         True      
____________________________________________________________________________
                     16 x 3              
Embedding                                 12         True      
____________________________________________________________________________
                     16 x 4              
Embedding                                 24         True      
Embedding                                 24         True      
Embedding                                 28         True      
____________________________________________________________________________
                     16 x 3              
Embedding                                 12         True      
Embedding                                 12         True      
___

In [35]:
siamese_learner.freeze()
siamese_learner.fit(n_epoch=3, lr=10e-4)

epoch,train_loss,valid_loss,time
0,58.312202,32.501068,00:41
1,31.172941,13.311623,00:45
2,18.305069,12.860377,00:47


In [36]:
siamese_learner.unfreeze()
siamese_learner.fit(n_epoch=3, lr=10e-5)

epoch,train_loss,valid_loss,time
0,12.527887,11.070065,00:49
1,11.559202,11.525859,00:49
2,10.512881,13.335747,00:50


In [ ]:
# TODO: inspect margin, using log space grid search - .01, .1, .5, .75, 1., 10., 50. - hypothesis is that the margin is too low and space is collapsging
# TODO: inspect the LR manually - smaller learning rates - maybe fix learning rate across grid search; maybe use lr_min / 2 ?

In [ ]:
siamese_learner.save("snn_margin_20")